In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

# change current working directory to the root of the project
import os
os.chdir(os.path.dirname(os.getcwd()))

# Purpose
- Purpose of this notebook is to train an CNN + LSTM model

In [2]:
import warnings
from IPython.display import display
import joblib

import pandas as pd
import numpy as np

import optuna

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Flatten, Bidirectional, Conv1D, MaxPooling1D, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.backend import clear_session

# Load Data

In [ ]:
# load data using joblib
X_train = joblib.load("data/transformed/X_train.pkl")
X_val = joblib.load("data/transformed/X_val.pkl")
X_test = joblib.load("data/transformed/X_test.pkl")

y_train = joblib.load("data/transformed/y_train.pkl")
y_val = joblib.load("data/transformed/y_val.pkl")
y_test = joblib.load("data/transformed/y_test.pkl")

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

# Hyper-parameter Tuning

In [ ]:
timesteps = X_train.shape[1]
n_features = X_train.shape[2]
n_classes = len(np.unique(y_train))

In [ ]:
def objective(trial: optuna.trial.Trial) -> float:
    '''Takes in hyperparameters as input, and trains a model that computes accuracy on the validation set.'''
    clear_session()

    # define early stopping
    early_stop = EarlyStopping(monitor='val_loss',patience=10)

    # define model architecture
    model = Sequential()

    model.add(Conv1D(filters=trial.suggest_categorical("filters", [128, 128*2, 128*3, 128*4, 128*5]), kernel_size=3, activation='relu', input_shape=(timesteps,n_features)))
    
    model.add(MaxPooling1D(pool_size=2))

    model.add(Dropout(trial.suggest_uniform("dropout", 0.0, 0.5)))

    model.add(Conv1D(filters=trial.suggest_categorical("filters", [128, 128*2, 128*3, 128*4, 128*5]), kernel_size=3, activation='relu'))

    model.add(MaxPooling1D(pool_size=2))

    model.add(Dropout(trial.suggest_uniform("dropout", 0.0, 0.5)))

    model.add(TimeDistributed(Flatten()))

    model.add(
            LSTM(
                units=trial.suggest_categorical("units", [128, 128*2, 128*3, 128*4, 128*5]),
                return_sequences=True))
    
    model.add(
        LSTM(
            units=trial.suggest_categorical("units", [128, 128*2, 128*3, 128*4, 128*5])))

    model.add(Dropout(trial.suggest_uniform("dropout", 0.0, 0.5)))
    
    model.add(Dense(units=trial.suggest_categorical("units", [128, 128*2, 128*3, 128*4, 128*5]), activation='relu'))

    model.add(Dense(n_classes, activation='softmax'))

    # compile model
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer='adam',
        metrics=["sparse_categorical_accuracy"])
    
    # fit model
    model.fit(
        X_train,
        y_train,
        epochs=100,
        batch_size=32,
        validation_data=(X_val, y_val),
        shuffle=False,
        callbacks=[early_stop],
        verbose=False,
    )

    # compute validation error
    score = model.evaluate(X_val, y_val, verbose=0)
    
    return score[1]

In [ ]:
warnings.filterwarnings('ignore')

# optuna study
study = optuna.create_study(direction="maximize", study_name="cnn_lstm")
study.optimize(objective, n_trials=5)

In [ ]:
# print best parameters
best_params = study.best_trial.params
print(f'{best_params=}')

# Fit model with best parameters

In [ ]:
units = best_params['units']
dropout = best_params['dropout']
filters = best_params['filters']

In [ ]:
clear_session()
# define early stopping
early_stop = EarlyStopping(monitor='val_loss',patience=10)

# define model architecture
best_model = Sequential()

best_model.add(Conv1D(filters=filters, kernel_size=3, activation='relu', input_shape=(timesteps,n_features)))

best_model.add(Dropout(dropout))

best_model.add(MaxPooling1D(pool_size=2))

best_model.add(
        Bidirectional(LSTM(
            units=units,
            input_shape=(timesteps, n_features))))

best_model.add(
        Bidirectional(LSTM(
            units=units,
            input_shape=(timesteps, n_features))))

best_model.add(Dropout(dropout))

best_model.add(Dense(units=units))

best_model.add(Dense(n_classes, activation='softmax'))

# compile model
best_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer='adam',
    metrics=["sparse_categorical_accuracy"])

# fit model
best_model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_val, y_val),
    shuffle=False,
    callbacks=[early_stop],
    verbose=False,
)

In [ ]:
best_model.summary()

In [ ]:
losses = pd.DataFrame(best_model.history.history)
losses[['loss', 'val_loss']].plot()

In [ ]:
accuracy = pd.DataFrame(best_model.history.history)
accuracy[['sparse_categorical_accuracy', 'val_sparse_categorical_accuracy']].plot()

In [ ]:
# Evaluate the model accuracy on the test set.
score = best_model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

# Save Predictions & model

In [ ]:
# save the model using joblib
joblib.dump(best_model, "models/cnn_lstm.pkl")

In [ ]:
# save predictions
y_pred = best_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

y_pred = pd.DataFrame(y_pred, columns=["predicted_activity"])
y_pred.to_csv("predictions/predictions_cnn_lstm.csv", index=False)

In [ ]:
# https://link.springer.com/article/10.1007/s11370-021-00358-7 (CNN + LSTM)
# https://ieeexplore.ieee.org/document/7881728 (CNN)
# https://medium.com/@tanmaychauhan111/human-activity-recognition-using-lstm-cnn-8ccb1a42cb81
# user mcnemars test to compare models